# **Laboratorio 11: LLM y Agentes Autónomos 🤖**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### **Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Joaquín Herrera Suárez
- Nombre de alumno 2: Hecmar Taucare Reyes

### **Link de repositorio de GitHub:** [Repositorio](https://github.com/Joaquin-HS/MDS7202)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [ ]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [ ]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.

El ambiente Blackjack-v1 modela el clásico juego de cartas Blackjack, donde el objetivo es vencer al dealer logrando que la suma de las cartas propias se acerque más a 21, sin excederlo. El juego tiene una formulación como un Proceso de Decisión de Markov (MDP), definido por los siguientes componentes:

1. Estados: \\
Representados como una tupla (player_sum, dealer_card, usable_ace):
  - player_sum (4 - 21): La suma actual de las cartas del jugador.
  - dealer_card (1 - 10): El valor de la carta visible del dealer.
  - usable_ace (0 o 1): Indica si el jugador tiene un as que puede contarse como 11 sin exceder 21.

2. Acciones: \\
El espacio de acciones es Discrete(2) con las siguientes opciones:
  - 0 (Stick): Detenerse y no tomar más cartas.
  - 1 (Hit): Pedir una carta adicional.

3. Recompensas: \\
Las recompensas están definidas según el resultado del juego:
  - +1: Victoria del jugador.
  - -1: Derrota del jugador.
  - 0: Empate.
  - +1.5: Victoria con un blackjack natural (si natural=True).

4. Dinámica de transición: \\
Si el jugador pide una carta (Hit), se actualiza su estado sumando el valor de la nueva carta. Si supera 21, el juego termina con una recompensa de -1.
Si el jugador se detiene (Stick), el dealer juega hasta alcanzar una suma mínima de 17. Luego, se compara la suma final del dealer con la del jugador para determinar el resultado.

5. Estado inicial: \\
El juego comienza con una suma inicial para el jugador entre 4 y 21, una carta visible del dealer entre 1 y 10, y un indicador de si el jugador tiene un as utilizable.

6. Fin del juego: \\
El juego termina cuando, o bien el jugador supera 21 (bust), o cuando el jugador decide detenerse (Stick) y se evalúa el resultado contra el dealer.

#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

In [ ]:
import gymnasium as gym
import numpy as np

# Número de simulaciones
n_episodes = 5000
rewards = []

# Se simulan 5000 episodios
for _ in range(n_episodes):
    observation, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Se selecciona una acción aleatoria
        action = env.action_space.sample()
        observation, reward, done, _, _ = env.step(action)
        total_reward += reward

    rewards.append(total_reward)

# Se calculan las métricas
mean_reward = np.mean(rewards)
std_reward = np.std(rewards)

print(f"Promedio de recompensas: {mean_reward:.2f}")
print(f"Desviación estándar de recompensas: {std_reward:.2f}")

Promedio de recompensas: -0.38
Desviación estándar de recompensas: 0.90


Como era de esperar, el rendimiento de esta política es muy bajo, ya que no toma decisiones informadas.

Las recompensas obtenidas indican que la política aleatoria lleva a perder más partidas de las que se ganan (promedio negativo), y que se tiene una alta variabilidad en los resultados (desviación estándar alta), lo que es típico en un juego de azar como Blackjack.

#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

In [ ]:
'''
Se utilizará el algoritmo DQN (Deep Q-Network) por su efectividad para resolver
problemas con espacios de acción discretos como Blacjack (se utiliza una red neuronal
para aproximar la función Q (que estima el valor esperado de cada acción en un
estado dado)). Además, es menos complejo que otros algoritmos, pero lo suficientemente
avanzado como para mostrar una mejora significativa sobre el baseline.
'''

import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

# Se crea el modelo DQN
model = DQN("MlpPolicy", env, verbose=1, learning_rate=0.001, gamma=0.99, exploration_fraction=0.1, seed=42)

# Se entrena el modelo
model.learn(total_timesteps=10000)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.252    |
|    n_updates        | 1940     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.55     |
|    ep_rew_mean      | -0.03    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 5100     |
|    fps              | 330      |
|    time_elapsed     | 23       |
|    total_timesteps  | 7872     |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.178    |
|    n_updates        | 1942     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.53     |
|    ep_rew_mean      | -0.05    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         |

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [ ]:
# Se evalúa el modelo
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=5000, deterministic=True)

print(f"Recompensa promedio: {mean_reward:.2f} ± {std_reward:.2f}")

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Recompensa promedio: -0.08 ± 0.96


El modelo entrenado tiene un promedio de recompensas significativamente menos negativo (-0.08) que el baseline (-0.38). Esto indica que el modelo entrenado pierde menos partidas en promedio y se acerca a un rendimiento neutral. Por su parte, la desviación estándar del modelo entrenado (0.96) es ligeramente mayor que la del baseline (0.90). Esto sugiere que hay una mayor variabilidad en las recompensas obtenidas por el modelo entrenado. Esto podría deberse a la naturaleza estocástica del ambiente o a que el modelo todavía no ha alcanzado una política completamente óptima.

De esta forma, el modelo entrenado tiene un desempeño claramente mejor que el baseline, ya que logra reducir significativamente las pérdidas promedio.
Aunque no alcanza un rendimiento positivo, está demostrando que puede aprender estrategias más efectivas que las acciones completamente aleatorias. De todas formas, el modelo podría beneficiarse de aumentar su entrenamiento (mayor `total_timesteps`); optimizar los hiperparámetros: `learning_rate` (tasa de aprendizaje para el optimizador), `gamma` (factor de descuento para recompensas futuras), o `exploration_fraction` (fracción de tiempo donde el agente realiza exploración en lugar de explotación) para que el modelo converja mejor; o directamente comparar este resultado con otros algoritmos de RL.

#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [ ]:
# Se inspecciona el formato de un estado
initial_state, _ = env.reset()
print(f"Estado inicial: {initial_state}")
print(f"Tipo de estado: {type(initial_state)}")

Estado inicial: [12  6  1]
Tipo de estado: <class 'numpy.ndarray'>


Los estados pertenecen a la clase `numpy.ndarray`.

In [ ]:
def get_agent_action(model, state):
    """
    Obtiene la acción del agente para un estado dado.

    Args:
    - model: Modelo entrenado (DQN).
    - state: Estado del ambiente (tuple o array).

    Returns:
    - Acción seleccionada por el modelo (int).
    """
    # Se convierte el estado al formato esperado por el modelo (array)
    if isinstance(state, tuple):
        state = np.array(state, dtype=np.float32)

    # Se selecciona la acción usando el modelo
    action, _ = model.predict(state, deterministic=True)
    return action

# Escenarios propuestos
scenarios = [
    (6, 7, 0),  # Suma de cartas = 6, Dealer muestra 7, sin as
    (19, 3, 1)  # Suma de cartas = 19, Dealer muestra 3, con as
]

# Se evalúan las acciones del agente en cada escenario
for i, state in enumerate(scenarios):
    action = get_agent_action(model, state)
    action_str = "Pedir carta (Hit)" if action == 1 else "Detenerse (Stick)"
    print(f"Escenario {i+1}: Estado = {state}, Acción = {action_str}")

Escenario 1: Estado = (6, 7, 0), Acción = Pedir carta (Hit)
Escenario 2: Estado = (19, 3, 1), Acción = Detenerse (Stick)


Dado el estado del escenario 1, lo esperado es que el agente pida una carta (Hit), ya que tiene pocas probabilidades de superar 21. Por otro lado, para el escenario 2, lo esperado es que el agente se detenga (Stick), ya que tiene una suma segura frente a la carta del dealer y pedir una carta sería muy arriesgado. Como se puede ver en "print", el agente toma acciones coherentes con las reglas del juego.

### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


In [ ]:
import gymnasium as gym
env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [ ]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

En primer lugar, se destaca que especificar el parámetro `continuous = True` implica que el espacio de acciones del ambiente cambia de ser discreto a continuo.

El ambiente LunarLander consiste en un problema clásico de optimización de trayectoria donde el objetivo es aterrizar una nave espacial en una plataforma situada en el centro de un mapa 2D.

1. Estados: \\
  El espacio de observaciones es un vector continuo de 8 dimensiones (Box), que describe:

    - Posición de la nave: coordenadas $x$ e $y$
    - Velocidades lineales: $v_x$ y $v_y$
    - Ángulo de orientación ($\theta$)
    - Velocidad angular ($\omega$)
    - Contacto de las patas de la nave con el suelo (2 valores booleanos).

  Un ejemplo de estado se puede ver a continuación: $[x, y, v_x, v_y, θ, ω, leg1_{contact}, leg2_{contact}]$

2. Acciones: \\
  En modo discreto:
    - 0: Hacer nada.
    - 1: Encender el motor izquierdo.
    - 2: Encender el motor principal.
    - 3: Encender el motor derecho.

  En modo continuo (el que se tiene al haber definido `continuous = True`):
  un vector de dos elementos:
    - $a[0]$: Potencia del motor principal (-1 a 1)
    - $a[1]$: Potencia de los propulsores laterales (-1 a 1).

3. Recompensas: \\
  Las recompensas están diseñadas para incentivar un aterrizaje seguro:

  - Proximidad al objetivo y velocidad reducida: recompensa positiva.
  - Contacto de las patas con el suelo: +10 puntos por pata.
  - Movimiento en exceso, inclinación o uso de combustible: penalización.
  - Aterrizaje seguro: +100 puntos.
  - Colisión: -100 puntos.
  - El episodio se considera resuelto si el puntaje total es $\geq 200$.

4. Condiciones de finalización \\
Un episodio termina si:

  - La nave colisiona con la superficie lunar.
  - La nave sale de los límites del mapa.
  - La nave queda inmóvil.

Se destaca que en LunarLander con `continuous=True`, las acciones son vectores continuos que permiten mayor control con respecto al caso de Blackjack, donde las acciones son simplemente binarias: "hit" (pedir carta) o "stick" (detenerse).

#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

In [ ]:
import numpy as np

# Lista para almacenar las recompensas de cada episodio
episodic_rewards = []

# Número de episodios
num_episodes = 10

# Se simula el ambiente con acciones aleatorias
for episode in range(num_episodes):
    state, _ = env.reset()
    total_reward = 0
    done = False

    while not done:
        # Se selecciona una acción aleatoria dentro del espacio de acción continuo
        random_action = env.action_space.sample()
        state, reward, done, _, _ = env.step(random_action)
        total_reward += reward

    episodic_rewards.append(total_reward)

# Se calculan las métricas
mean_reward = np.mean(episodic_rewards)
std_reward = np.std(episodic_rewards)

print(f"Recompensa promedio: {mean_reward:.2f}")
print(f"Desviación estándar de las recompensas: {std_reward:.2f}")

Recompensa promedio: -243.93
Desviación estándar de las recompensas: 108.96


El performance de esta política es bajo (`recompensa promedio: -243.93`), ya que no sigue ninguna estrategia para optimizar la posición, velocidad o el uso de motores, lo que tiende a resultar en posibles colisiones o aterrizajes incorrectos. Por su parte, por el carácter aleatorio de las acciones, los resultados de cada episodio pueden tener grandes variaciones, lo que se aprecia en el alto valor de desviación estándar obtenido (`108.96`).

#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [ ]:
'''
Se utilizará el algoritmo PPO (Proximal Policy Optimization) por su efectividad
y estabilidad para resolver problemas con espacios de acción continuos como el
LunarLander en su configuración continuous=True, al modelar distribuciones
gaussianas para las acciones.
'''

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Se crea el modelo PPO
model = PPO("MlpPolicy", env, verbose=1, learning_rate=0.0003, gamma=0.99, seed=42)

# Se entrena el modelo con 10000 timesteps
model.learn(total_timesteps=10000)

# Se guarda el modelo entrenado
model.save("ppo_lunarlander")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 111      |
|    ep_rew_mean     | -214     |
| time/              |          |
|    fps             | 653      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 120         |
|    ep_rew_mean          | -232        |
| time/                   |             |
|    fps                  | 492         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.004595261 |
|    clip_fraction        | 0.0311      |
|    clip_range           | 0.2         |
|    entropy_loss   

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [ ]:
# Se carga el modelo entrenado
model = PPO.load("ppo_lunarlander")

# Lista para almacenar las recompensas de cada episodio
episodic_rewards = []

# Número de episodios para la evaluación
num_episodes = 10

# Se evalúa el modelo
for episode in range(num_episodes):
    state, _ = env.reset()
    total_reward = 0
    done = False

    while not done:
        # Acción del modelo entrenado
        action, _ = model.predict(state)
        state, reward, done, _, _ = env.step(action)
        total_reward += reward

    episodic_rewards.append(total_reward)

# Se calculan las métricas
mean_reward = np.mean(episodic_rewards)
std_reward = np.std(episodic_rewards)

print(f"Recompensa promedio: {mean_reward:.2f}")
print(f"Desviación estándar de las recompensas: {std_reward:.2f}")

Recompensa promedio: -142.87
Desviación estándar de las recompensas: 96.47


El modelo entrenado tiene un promedio de recompensa significativamente menos negativo (-142.87) que el baseline (-243.93). Esto indica que el modelo entrenado ha aprendido a realizar acciones más orientadas hacia el objetivo, como acercarse al área de aterrizaje y mantener la nave en buenas condiciones. Por su parte, la desviación estándar del modelo entrenado (96.47) es ligeramente menor que la del baseline (108.96), lo que sugiere un comportamiento más consistente en las políticas aprendidas, aunque todavía con cierta variabilidad presente.

De esta forma, el modelo entrenado tiene un desempeño claramente mejor que el baseline, ya que aprendió a evitar comportamientos altamente penalizados, es decir, una estrategia más efectiva que efectuar acciones completamente aleatorias. Ahora bien, es importante destacar que todavía no alcanza un desempeño que resuelva completamente el problema. De igual forma que antes, el modelo podría beneficiarse de aumentar su entrenamiento (mayor `total_timesteps`); optimizar los hiperparámetros: `learning_rate` (tasa de aprendizaje para el optimizador) o `gamma` (factor de descuento para recompensas futuras) para que el modelo converja mejor; o directamente comparar este resultado con otros algoritmos de RL.

#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

In [ ]:
# Stable-Baselines3 espera trabajar con un entorno envuelto en un VecEnv
# (un vector de entornos para manejar múltiples episodios en paralelo)

from stable_baselines3.common.env_util import DummyVecEnv

vec_env = DummyVecEnv([lambda: env])

def evaluate_model(env, model, n_episodes=10):
    rewards = []
    for episode in range(n_episodes):
        obs = env.reset()
        done = False
        episode_reward = 0
        while not done:
            action, _ = model.predict(obs)
            obs, reward, done, info = env.step(action)
            episode_reward += reward
        rewards.append(episode_reward)

    avg_reward = np.mean(rewards)
    std_reward = np.std(rewards)
    print(f"Recompensa promedio: {avg_reward:.2f}")
    print(f"Desviación estándar de las recompensas: {std_reward:.2f}")
    return avg_reward, std_reward

##### learning_rate

Se disminuye la tasa de aprendizaje para estabilizar el entrenamiento en un ambiente complejo como el de LunarLander.

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Se crea el modelo PPO
model_learning_rate = PPO("MlpPolicy", env, verbose=1, learning_rate=0.0001, gamma=0.99, seed=42)

# Se entrena el modelo con 10000 timesteps
model_learning_rate.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 111      |
|    ep_rew_mean     | -214     |
| time/              |          |
|    fps             | 880      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 119          |
|    ep_rew_mean          | -223         |
| time/                   |              |
|    fps                  | 652          |
|    iterations           | 2            |
|    time_elapsed         | 6            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0027952273 |
|    clip_fraction        | 0.00703      |
|    clip_range           | 0.2          |
|    en

In [ ]:
# Evaluación del modelo optimizado
evaluate_model(vec_env, model_learning_rate)

Recompensa promedio: -153.18
Desviación estándar de las recompensas: 89.44


(-153.1816, 89.43967)

##### batch_size

Se aumenta el tamaño del batch dado el ambiente altamente complejo y continuo que es LunarLander.

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Se crea el modelo PPO
model_batch_size = PPO("MlpPolicy", env, verbose=1, batch_size=256, learning_rate=0.0003, gamma=0.99, seed=42)

# Se entrena el modelo con 10000 timesteps
model_batch_size.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 111      |
|    ep_rew_mean     | -214     |
| time/              |          |
|    fps             | 916      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 123          |
|    ep_rew_mean          | -229         |
| time/                   |              |
|    fps                  | 763          |
|    iterations           | 2            |
|    time_elapsed         | 5            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0038873914 |
|    clip_fraction        | 0.0169       |
|    clip_range           | 0.2          |
|    en

In [ ]:
# Evaluación del modelo optimizado
evaluate_model(vec_env, model_batch_size)

Recompensa promedio: -149.01
Desviación estándar de las recompensas: 91.98


(-149.0068, 91.97724)

##### total_timesteps

Se decide aumentar el número de pasos de entrenamiento para que PPO tenga más tiempo para aprender las políticas complejas de LunarLander.

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Se crea el modelo PPO
model_timesteps100000 = PPO("MlpPolicy", env, verbose=1, learning_rate=0.0003, gamma=0.99, seed=42)

# Se entrena el modelo con 100000 timesteps
model_timesteps100000.learn(total_timesteps=100000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 111      |
|    ep_rew_mean     | -214     |
| time/              |          |
|    fps             | 749      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 120         |
|    ep_rew_mean          | -232        |
| time/                   |             |
|    fps                  | 531         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.004595261 |
|    clip_fraction        | 0.0311      |
|    clip_range           | 0.2         |
|    entropy_loss   

In [ ]:
# Evaluación del modelo optimizado
evaluate_model(vec_env, model_timesteps100000)

Recompensa promedio: 99.12
Desviación estándar de las recompensas: 38.09


(99.12029, 38.09349)

In [ ]:
export_gif(model_timesteps100000)

![Comportamiento del agente entrenado](agent_performance.gif)

Como se puede observar en el gif, la nave tiene, en general, un comportamiento más refinado al ajustar su orientación y acelerar de manera controlada para aterrizar en la plataforma. Esto, además, se aprecia en el nivel de recompensa promedio obtenido, que cumple con ser mayor a 50, mostrando que el agente aprende una política eficiente.

Se destaca la relevancia de aumentar el número de pasos de entrenamiento para generar un modelo con mejor desempeño, por sobre realizar variaciones en el resto de hiperparámetros vistos. Sin embargo, también se menciona el aumento en el tiempo de entrenamiento, pasando de 15-20 segundos a 6 minutos, aproximadamente.

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

Enter your Google AI API key: ··········
Enter your Tavily API key: ··········


### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [ ]:
%pip install --upgrade --quiet PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [ ]:
import os
import requests
import PyPDF2

# Links a los documentos en GitHub
github_links = [
    "https://github.com/GAMES-UChile/Curso-Estadistica/raw/master/notas_de_clase.pdf",
    "https://github.com/GAMES-UChile/Curso-Aprendizaje-de-Maquinas/raw/master/notas_de_clase.pdf"
]

# Paths locales como alternativa
local_doc_paths = [
    "Notas_de_clase_Aprendizaje_de_Maquinas.pdf",
    "Notas_de_clase_Estadistica.pdf"
]

# Lista donde se almacenarán los paths a los documentos accesibles
doc_paths = []

# Se intentan descargar los archivos desde GitHub en primer lugar
for url in github_links:
    try:
        response = requests.get(url)
        if response.status_code == 200:
            # Se extrae el identificador único del URL
            identifier = url.split("/")[-4]  # Toma el segmento del curso (ejemplo: 'Curso-Estadistica') para diferenciar pdf's
            filename = f"{identifier}.pdf"
            with open(filename, "wb") as f:
                f.write(response.content)
            doc_paths.append(filename)
            print(f"Descargado y renombrado: {filename}")
        else:
            print(f"No se pudo descargar desde GitHub: {url} (Status: {response.status_code})")
    except Exception as e:
        print(f"Error al descargar {url}: {e}")

# Si no se pudieron descargar desde GitHub, se usan los archivos locales
# (se adjuntarán en la tarea entregada por UCursos, mas no en el repositorio).
if len(doc_paths) < 2:
    print("Intentando usar archivos locales...")
    for local_path in local_doc_paths:
        if os.path.exists(local_path):
            doc_paths.append(local_path)
            print(f"Archivo local encontrado: {local_path}")
        else:
            print(f"Archivo local no encontrado: {local_path}")

# Se verifica que se tienen al menos 2 documentos
assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

# Se verifica que el total de páginas es al menos 50
total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

print(f"Total de páginas: {total_paginas}")

Descargado y renombrado: Curso-Estadistica.pdf
Descargado y renombrado: Curso-Aprendizaje-de-Maquinas.pdf
Total de páginas: 274


#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.4 MB/s eta 0:00:00


In [ ]:
%pip install --upgrade --quiet faiss-cpu langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00


In [ ]:
import os
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

def extract_text_from_pdf(pdf_path):
    """Extrae texto de todas las páginas de un archivo PDF."""
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text()
    return text

def create_documents_from_pdfs(doc_paths):
    """Convierte PDFs a objetos Document para procesar con LangChain."""
    documents = []
    for doc_path in doc_paths:
        text = extract_text_from_pdf(doc_path)
        documents.append(Document(page_content=text, metadata={"source": doc_path}))
    return documents

def split_and_vectorize_documents(documents):
    """Divide los documentos en chunks y los vectoriza."""
    # Se inicializa el splitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    splits = text_splitter.split_documents(documents)  # Se divide en chunks

    print(f"Total de chunks creados: {len(splits)}")

    # Se crean los embeddings
    embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)  # Vectorización y almacenamiento

    return vectorstore

/usr/local/lib/python3.10/dist-packages/google/colab/html/_background_server.py:103: DeprecationWarning: make_current is deprecated; start the event loop first
  ioloop.make_current()


In [ ]:
# Se crean documentos desde PDFs
documents = create_documents_from_pdfs(doc_paths)

# Se dividen y vectorizan
vectorstore = split_and_vectorize_documents(documents)

# Se almacena la base de datos FAISS
faiss_file = "vectorstore.faiss"
vectorstore.save_local(faiss_file)
print(f"Vectorstore almacenado en {faiss_file}")

Total de chunks creados: 1362
Vectorstore almacenado en vectorstore.faiss


#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI

# Se configura un modelo de generación de texto
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # Modelo de lenguaje
    temperature=0, # Probabilidad de "respuestas creativas"
    max_tokens=None, # Sin tope de tokens
    timeout=None, # Sin timeout
    max_retries=2, # Número máximo de intentos
)

# Se crea la chain RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    verbose=True
)

#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

In [ ]:
# Listado de preguntas y respuestas correctas (tuplas)
qa_pairs = [
    ("¿Qué es un estimador de máxima verosimilitud?", "Un estimador de máxima verosimilitud es un método para estimar los parámetros de un modelo probabilístico, buscando los valores que hacen que los datos observados sean más probables bajo ese modelo."),
    ("¿Qué es el aprendizaje supervisado?", "El aprendizaje supervisado es un tipo de aprendizaje automático donde el modelo es entrenado con datos etiquetados para hacer predicciones o clasificaciones."),
    ("¿Qué es un modelo de regresión lineal?", "Un modelo de regresión lineal es un tipo de modelo estadístico que busca establecer una relación lineal entre una variable dependiente y una o más variables independientes.")
]

# Se muestran las respuestas para cada pregunta
for question, correct_answer in qa_pairs:
    print(f"Pregunta: {question}")
    print(f"Respuesta correcta: {correct_answer}")

    # Se usa la solución RAG para obtener la respuesta
    response = qa_chain.run(question)

    print(f"Respuesta generada por RAG: {response}")

Pregunta: ¿Qué es un estimador de máxima verosimilitud?
Respuesta correcta: Un estimador de máxima verosimilitud es un método para estimar los parámetros de un modelo probabilístico, buscando los valores que hacen que los datos observados sean más probables bajo ese modelo.


> Entering new RetrievalQA chain...

> Finished chain.
Respuesta generada por RAG: Según el texto proporcionado, el estimador de máxima verosimilitud (EMV) es una función de los datos que busca entregar un valor cercano a un parámetro desconocido.  Se basa en la función de verosimilitud, que representa la probabilidad de observar los datos dados un valor específico del parámetro.  El EMV se define como el valor del parámetro que maximiza la función de verosimilitud (o una función monótonamente creciente de esta).  En otras palabras, el EMV es el valor del parámetro que hace más probable la observación de los datos.

Pregunta: ¿Qué es el aprendizaje supervisado?
Respuesta correcta: El aprendizaje supervisado es un 

En general, las respuestas entregadas por la solución RAG son buenas. En todas ellas se explaya más de lo que se aprecia en la tupla correcta, lo que tiene sentido ya que está sacando la información de notas de clase de un curso con alto enfoque matemático.

Ahora bien, se destaca que la segunda respuesta pareciera no concluír del todo, lo que puede deberse a varias razones: por un lado, el alto volumen de datos y/o complejidad de la pregunta podría estar generando un corte prematuro debido a limitaciones de tiempo; por otro, el valor de `temperature` definido es 0, lo que significa que el modelo está generando respuestas determinísticas, es decir, más directas y menos variadas, lo que implica que quizás no está explorando de manera apropiada para completar las respuestas de mejor forma; y por último, que el modelo esté tratando de generar una respuesta a partir de fragmentos que no contienen toda la información necesaria, o directamente presentan problemas al hacer la recuperación de los documentos, generando un atasque en el modelo y, así, una respuesta incompleta.

#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

Para comparar desempeños se obtuvieron los parámetros por defecto (ejecutados en la iteración anterior) del siguiente [link](https://python.langchain.com/api_reference/_modules/langchain_core/vectorstores/base.html#VectorStore.as_retriever).


##### Cantidad de chunks recuperados

El número de chunks que recupera el modelo depende de la cantidad de fragmentos que el recuperador de documentos (FAISS) retorna durante la búsqueda (por defecto son 4). El recuperar demasiados chunks puede generar que el modelo se sobrecargue y no logre identificar los fragmentos más relevantes. Por otra parte, si se recuperan pocos chunks, puede que no tenga suficiente información para responder adecuadamente a la pregunta.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI

# Se configura un modelo de generación de texto
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # Modelo de lenguaje
    temperature=0, # Probabilidad de "respuestas creativas"
    max_tokens=None, # Sin tope de tokens
    timeout=None, # Sin timeout
    max_retries=2, # Número máximo de intentos
)

# Se crea la chain RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 16}),  # Se recuperan 16 chunks
    verbose=True
)

# Listado de preguntas y respuestas correctas (tuplas)
qa_pairs = [
    ("¿Qué es un estimador de máxima verosimilitud?", "Un estimador de máxima verosimilitud es un método para estimar los parámetros de un modelo probabilístico, buscando los valores que hacen que los datos observados sean más probables bajo ese modelo."),
    ("¿Qué es el aprendizaje supervisado?", "El aprendizaje supervisado es un tipo de aprendizaje automático donde el modelo es entrenado con datos etiquetados para hacer predicciones o clasificaciones."),
    ("¿Qué es un modelo de regresión lineal?", "Un modelo de regresión lineal es un tipo de modelo estadístico que busca establecer una relación lineal entre una variable dependiente y una o más variables independientes.")
]

# Se muestran las respuestas para cada pregunta
for question, correct_answer in qa_pairs:
    print(f"Pregunta: {question}")
    print(f"Respuesta correcta: {correct_answer}")

    # Se usa la solución RAG para obtener la respuesta
    response = qa_chain.run(question)

    print(f"Respuesta generada por RAG: {response}")

Pregunta: ¿Qué es un estimador de máxima verosimilitud?
Respuesta correcta: Un estimador de máxima verosimilitud es un método para estimar los parámetros de un modelo probabilístico, buscando los valores que hacen que los datos observados sean más probables bajo ese modelo.


> Entering new RetrievalQA chain...

> Finished chain.
Respuesta generada por RAG: Un estimador de máxima verosimilitud (EMV) es una función de los datos que busca encontrar el valor de un parámetro desconocido que maximiza la probabilidad de haber observado los datos.  En otras palabras, se busca el valor del parámetro que hace más probable la muestra obtenida.  Se puede definir con respecto a la función de verosimilitud o a cualquier función no decreciente de esta.  A menudo, se maximiza la log-verosimilitud en lugar de la verosimilitud para facilitar los cálculos.

Pregunta: ¿Qué es el aprendizaje supervisado?
Respuesta correcta: El aprendizaje supervisado es un tipo de aprendizaje automático donde el modelo es

Como se puede ver, en general las respuestas son más completas y precisas con respecto a las que se tenían en la sección anterior. Además, ahora la segunda respuesta si logró terminar de generarse, lo que permite comprobar, al menos en parte, una de las razones dadas previamente de porqué no había concluído. Se destaca que el modelo no se sobrecargó en este caso.

##### Tipo de búsqueda

El tipo de búsqueda se refiere al algoritmo utilizado para recuperar los documentos más relevantes. Por un lado, el método `similarity` (el que está definido por defecto) busca y devuelve los k documentos más similares en función de la distancia en el espacio de embeddings (como la distancia coseno o euclidiana). Por otro, el método `mmr` (Maximal Marginal Relevance) equilibra la relevancia y la diversidad de los documentos recuperados, lo que es útil si se quiere evitar que se devuelvan múltiples chunks con contenido muy similar. Por último, el método `similarity_score_threshold` solo devuelve documentos cuya puntuación de similitud supere un umbral definido por el parámetro score_threshold.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI

# Se configura un modelo de generación de texto
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # Modelo de lenguaje
    temperature=0, # Probabilidad de "respuestas creativas"
    max_tokens=None, # Sin tope de tokens
    timeout=None, # Sin timeout
    max_retries=2, # Número máximo de intentos
)

# Se crea la chain RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.8}),
    verbose=True
)

# Listado de preguntas y respuestas correctas (tuplas)
qa_pairs = [
    ("¿Qué es un estimador de máxima verosimilitud?", "Un estimador de máxima verosimilitud es un método para estimar los parámetros de un modelo probabilístico, buscando los valores que hacen que los datos observados sean más probables bajo ese modelo."),
    ("¿Qué es el aprendizaje supervisado?", "El aprendizaje supervisado es un tipo de aprendizaje automático donde el modelo es entrenado con datos etiquetados para hacer predicciones o clasificaciones."),
    ("¿Qué es un modelo de regresión lineal?", "Un modelo de regresión lineal es un tipo de modelo estadístico que busca establecer una relación lineal entre una variable dependiente y una o más variables independientes.")
]

# Se muestran las respuestas para cada pregunta
for question, correct_answer in qa_pairs:
    print(f"Pregunta: {question}")
    print(f"Respuesta correcta: {correct_answer}")

    # Se usa la solución RAG para obtener la respuesta
    response = qa_chain.run(question)

    print(f"Respuesta generada por RAG: {response}")

Pregunta: ¿Qué es un estimador de máxima verosimilitud?
Respuesta correcta: Un estimador de máxima verosimilitud es un método para estimar los parámetros de un modelo probabilístico, buscando los valores que hacen que los datos observados sean más probables bajo ese modelo.


> Entering new RetrievalQA chain...



> Finished chain.
Respuesta generada por RAG: Un estimador de máxima verosimilitud (EMV) es un método para estimar los parámetros de una distribución de probabilidad dada una muestra de datos.  En esencia, busca los valores de los parámetros que maximizan la probabilidad de observar los datos que se han obtenido.  Es decir, encuentra los parámetros que hacen que los datos observados sean lo más probables posible.

Pregunta: ¿Qué es el aprendizaje supervisado?
Respuesta correcta: El aprendizaje supervisado es un tipo de aprendizaje automático donde el modelo es entrenado con datos etiquetados para hacer predicciones o clasificaciones.


> Entering new RetrievalQA chain...



> Finished chain.
Respuesta generada por RAG: El aprendizaje supervisado es un tipo de aprendizaje automático donde un algoritmo aprende de un conjunto de datos etiquetados.  Esto significa que cada dato en el conjunto de entrenamiento está asociado con una etiqueta o salida correcta.  El algoritmo aprende a mapear las entradas a las salidas correctas, y luego puede usar este conocimiento para predecir las salidas de nuevas entradas que no ha visto antes.

Pregunta: ¿Qué es un modelo de regresión lineal?
Respuesta correcta: Un modelo de regresión lineal es un tipo de modelo estadístico que busca establecer una relación lineal entre una variable dependiente y una o más variables independientes.


> Entering new RetrievalQA chain...

> Finished chain.
Respuesta generada por RAG: Un modelo de regresión lineal es una herramienta estadística que se utiliza para modelar la relación entre una variable dependiente (la que se quiere predecir) y una o más variables independientes (las que se ut

De las respuestas generadas se destaca la gran relevancia que tiene el nivel de exigencia con respecto al umbral de similitud permitido, sobre todo para preguntas que no son tan directas de responder al requerir un conocimiento más amplio. Además, se evidencia como ahora la última pregunta tuvo una respuesta mucho más completa (en un sentido matemático) y apropiada/entendible (la respuesta a esta pregunta en la sección anterior tenía una parte matemática difícil de entender en un primer y rápido análisis), lo que se entiende por el alto requisito mínimo de similitud.

##### Tamaño del chunk (chunk_size y chunk_overlap)

El tamaño del chunk es el número de tokens que contiene cada fragmento de texto extraído de los documentos. Por una parte, el hecho de que los chunks sean más pequeños puede permitir que el modelo enfoque su atención en partes más específicas del texto, pero pueden perder contexto si el fragmento no cubre información clave. Por otro lado, los chunks más grandes pueden permitir una mejor cobertura de contexto, pero también pueden hacer que el modelo se enfoque en demasiada información a la vez, lo que podría resultar en respuestas más difusas o imprecisas.

In [58]:
def split_and_vectorize_documents(documents):
    """Divide los documentos en chunks y los vectoriza."""
    # Se inicializa el splitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)  # Se disminuye chunk_size y chunk_overlap
    splits = text_splitter.split_documents(documents)  # Se divide en chunks

    print(f"Total de chunks creados: {len(splits)}")

    # Se crean los embeddings
    embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)  # Vectorización y almacenamiento

    return vectorstore

In [59]:
# Se crean documentos desde PDFs
documents = create_documents_from_pdfs(doc_paths)

# Se dividen y vectorizan
vectorstore = split_and_vectorize_documents(documents)

# Se almacena la base de datos FAISS
faiss_file = "vectorstore.faiss"
vectorstore.save_local(faiss_file)
print(f"Vectorstore almacenado en {faiss_file}")

Total de chunks creados: 4127
Vectorstore almacenado en vectorstore.faiss


In [60]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI

# Se configura un modelo de generación de texto
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # Modelo de lenguaje
    temperature=0, # Probabilidad de "respuestas creativas"
    max_tokens=None, # Sin tope de tokens
    timeout=None, # Sin timeout
    max_retries=2, # Número máximo de intentos
)

# Se crea la chain RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    verbose=True
)

# Listado de preguntas y respuestas correctas (tuplas)
qa_pairs = [
    ("¿Qué es un estimador de máxima verosimilitud?", "Un estimador de máxima verosimilitud es un método para estimar los parámetros de un modelo probabilístico, buscando los valores que hacen que los datos observados sean más probables bajo ese modelo."),
    ("¿Qué es el aprendizaje supervisado?", "El aprendizaje supervisado es un tipo de aprendizaje automático donde el modelo es entrenado con datos etiquetados para hacer predicciones o clasificaciones."),
    ("¿Qué es un modelo de regresión lineal?", "Un modelo de regresión lineal es un tipo de modelo estadístico que busca establecer una relación lineal entre una variable dependiente y una o más variables independientes.")
]

# Se muestran las respuestas para cada pregunta
for question, correct_answer in qa_pairs:
    print(f"Pregunta: {question}")
    print(f"Respuesta correcta: {correct_answer}")

    # Se usa la solución RAG para obtener la respuesta
    response = qa_chain.run(question)

    print(f"Respuesta generada por RAG: {response}")

Pregunta: ¿Qué es un estimador de máxima verosimilitud?
Respuesta correcta: Un estimador de máxima verosimilitud es un método para estimar los parámetros de un modelo probabilístico, buscando los valores que hacen que los datos observados sean más probables bajo ese modelo.


> Entering new RetrievalQA chain...

> Finished chain.
Respuesta generada por RAG: Based on the provided text, a maximum likelihood estimator (EMV) is an estimator that finds an estimator based on how probable it is that the estimator generated the data.  The text also mentions that the maximum likelihood estimator is θMV=max{xi}n, but doesn't fully explain what that means in a broader context.

Pregunta: ¿Qué es el aprendizaje supervisado?
Respuesta correcta: El aprendizaje supervisado es un tipo de aprendizaje automático donde el modelo es entrenado con datos etiquetados para hacer predicciones o clasificaciones.


> Entering new RetrievalQA chain...

> Finished chain.
Respuesta generada por RAG: El aprendizaje 

Las respuestas generadas coinciden con lo mencionado sobre si se reducía el tamaño del chunk: en general, se pierde contexto cuando el fragmento no cuenta con información clave. El modelo no logra generar respuestas del todo satisfactorias, ya sea porque menciona que el texto no explica de manera completa ciertos aspectos; no entrega la información de manera completa o bien explicada; o directamente experimenta un cambio de idioma, probablemente porque no tenía en su contexto el que la pregunta era en español.

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

In [61]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Se configura la tool Tavily para devolver un máximo de 3 resultados por búsqueda
tavily_tool = TavilySearchResults(max_results=3)

#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

In [64]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=f29e7761509f0fc7b21d0b9656936848b7925a20ff553c7f5804c4200ab30b6a
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [65]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

# Se inicializa el wrapper de la API de Wikipedia
wikipedia_api_wrapper = WikipediaAPIWrapper()

# Se crea la tool
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia_api_wrapper)

#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

In [67]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate
from langchain import hub

# Prompt ReAct predefinido de LangChain
react_prompt = hub.pull("hwchase17/react")

# Lista de herramientas
tools = [tavily_tool, wikipedia_tool]

# Agente ReAct
agent = create_react_agent(llm, tools, react_prompt)

# Se convierte el agente en un executor para usarlo
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

In [68]:
# Pregunta que debería usar Wikipedia
response = agent_executor.invoke({"input": "¿Quién escribió Don Quijote de la Mancha?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: This question asks who wrote Don Quixote.  I should consult Wikipedia, as it's a well-known work of literature and Wikipedia is a reliable source for this type of information.

Action: wikipedia
Action Input: Don Quixote
Page: Don Quixote
Summary: Don Quixote, the full title being The Ingenious Gentleman Don Quixote of La Mancha, is a Spanish novel by Miguel de Cervantes. It was originally published in two parts, in 1605 and 1615. Considered a founding work of Western literature, it is often said to be the first modern novel. Don Quixote is also one of the most-translated books in the world and one of the best-selling novels of all time. 
The plot revolves around the adventures of a member of the lowest nobility, an hidalgo from La Mancha named Alonso Quijano, who reads so many chivalric romances that he loses his mind and decides to become a knight-errant (caballero andante) to revive chivalry and serve his nation, under the name Don Qu

In [69]:
# Pregunta que debería usar Tavily
response = agent_executor.invoke({"input": "¿Cuáles son las últimas noticias sobre inteligencia artificial?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: To answer the question about the latest news on artificial intelligence, I need to use a search engine that provides current and reliable information.  Tavily_search_results_json seems best suited for this.

Action: tavily_search_results_json
Action Input: "latest news artificial intelligence"
[{'url': 'https://techcrunch.com/category/artificial-intelligence/', 'content': 'AI News & Artificial Intelligence | TechCrunch AI News & Artificial Intelligence | TechCrunch AI Site Search Toggle AI AI News coverage on artificial intelligence and machine learning tech, the companies building them, and the ethical issues AI raises today. AI AI AI AI AI AI AI Juna.ai wants to use AI agents to make factories more energy-efficient Norwegian startup Factiverse wants to fight disinformation with AI AI A popular technique to make AI more efficient has drawbacks AI AI AI AI Here’s the full list of 44 US AI startups that have raised $100M or more in 2024 O

El agente debería ocupar Tavily para preguntas sobre temas actuales o información dinámica que no suele estar en una enciclopedia, como noticias o eventos recientes. Por otra parte, debería ocupar Wikipedia para preguntas históricas, académicas o generales que tienen respuestas bien documentadas en su base de conocimiento.

De esta forma, según la verificación de respuestas realizadas se aprecia que el agente está usando correctamente las tools disponibles.

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

Se ejecutará de nuevo la solución RAG, utilizando la versión previa a la prueba de hiperparámetros:

In [70]:
import os
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

def extract_text_from_pdf(pdf_path):
    """Extrae texto de todas las páginas de un archivo PDF."""
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text()
    return text

def create_documents_from_pdfs(doc_paths):
    """Convierte PDFs a objetos Document para procesar con LangChain."""
    documents = []
    for doc_path in doc_paths:
        text = extract_text_from_pdf(doc_path)
        documents.append(Document(page_content=text, metadata={"source": doc_path}))
    return documents

def split_and_vectorize_documents(documents):
    """Divide los documentos en chunks y los vectoriza."""
    # Se inicializa el splitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    splits = text_splitter.split_documents(documents)  # Se divide en chunks

    print(f"Total de chunks creados: {len(splits)}")

    # Se crean los embeddings
    embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)  # Vectorización y almacenamiento

    return vectorstore

# Se crean documentos desde PDFs
documents = create_documents_from_pdfs(doc_paths)

# Se dividen y vectorizan
vectorstore = split_and_vectorize_documents(documents)

# Se almacena la base de datos FAISS
faiss_file = "vectorstore.faiss"
vectorstore.save_local(faiss_file)
print(f"Vectorstore almacenado en {faiss_file}")

Total de chunks creados: 1362
Vectorstore almacenado en vectorstore.faiss


In [71]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI

# Se configura un modelo de generación de texto
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # Modelo de lenguaje
    temperature=0, # Probabilidad de "respuestas creativas"
    max_tokens=None, # Sin tope de tokens
    timeout=None, # Sin timeout
    max_retries=2, # Número máximo de intentos
)

# Se crea la chain RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    verbose=True
)

Ahora sí, se generan las tools:

In [72]:
from langchain.tools import tool

@tool
def rag_tool(question: str) -> str:
    """
    Usa la solución RAG para responder preguntas basadas en el contenido de los PDFs.
    """
    response = qa_chain.run(question)
    return response

@tool
def react_agent_tool(input_text: str) -> str:
    """
    Usa el agente ReAct para responder preguntas usando Tavily y Wikipedia.
    """
    response = agent_executor.invoke({"input": input_text})
    return response["output"]

# Lista de herramientas transformadas
tools = [rag_tool, react_agent_tool]

#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

In [73]:
from langchain.prompts import PromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor

# Prompt para el agente supervisor
supervisor_prompt = """
Eres un agente supervisor con acceso a herramientas especializadas:
1. Usa `rag_tool` para responder preguntas relacionadas con información extraída de PDFs.
2. Usa `react_agent_tool` para responder preguntas que requieren búsquedas en la web.

Pregunta: {input}
{agent_scratchpad}
"""

# Agente supervisor
supervisor_agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,  # Lista de herramientas
    prompt=PromptTemplate.from_template(supervisor_prompt)
)

# Se convierte el agente en ejecutor
supervisor = AgentExecutor(agent=supervisor_agent, tools=tools, verbose=True)

#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

In [75]:
# Preguntas de prueba
questions = [
    "¿Qué es un estimador de máxima verosimilitud?",
    "¿Qué es el aprendizaje supervisado?",
    "¿Qué equipo ganó la última Eurocopa?"
]

# Se obtienen respuestas del supervisor
for question in questions:
    print(f"Pregunta: {question}")
    response = supervisor.invoke({"input": question})
    print(f"Respuesta del supervisor: {response['output']}")

Pregunta: ¿Qué es un estimador de máxima verosimilitud?


> Entering new AgentExecutor chain...

Invoking: `react_agent_tool` with `{'input_text': '¿Qué es un estimador de máxima verosimilitud?'}`




> Entering new AgentExecutor chain...
Thought: To answer this question, I need to understand what a maximum likelihood estimator is.  A Wikipedia search should provide a good definition and explanation.

Action: wikipedia
Action Input: Maximum likelihood estimation
Page: Maximum likelihood estimation
Summary: In statistics, maximum likelihood estimation (MLE) is a method of estimating the parameters of an assumed probability distribution, given some observed data. This is achieved by maximizing a likelihood function so that, under the assumed statistical model, the observed data is most probable. The point in the parameter space that maximizes the likelihood function is called the maximum likelihood estimate. The logic of maximum likelihood is both intuitive and flexible, and as such the 

El agente supervisor decide qué herramienta utilizar en función de la pregunta. Según lo esperado, para preguntas relacionadas con los PDFs entregados, debería usar `rag_tool`, y para preguntas que requieran buscar en la web, usará `react_agent_tool`. Ahora bien, como se puede ver, el agente siempre busca por Wikipedia, inclusive para aquellas preguntas que están muy relacionadas a los archivos PDF. Esto se debe principalmente a la generalidad de las preguntas realizadas, ya que basta con una búsqueda rápida en Wikipedia para responderlas. Ahora bien, si se hicieran preguntas mucho más específicas que su respuesta probablemente no esté en Wikipedia, como por ejemplo algunas relacionadas a un paper específico de la materia de estadística o aprendizaje de máquinas, probablemente sí vaya a buscar la respuesta utilizando `rag_tool`.

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

Un Router es un modelo explícitamente programado para clasificar preguntas y dirigirlas hacia la herramienta adecuada basándose en reglas predefinidas (como un clasificador de texto). En contraste, el enfoque con el agente supervisor utiliza un LLM para razonar dinámicamente sobre qué herramienta utilizar.

El enfoque del agente supervisor cuenta con la ventaja de que puede razonar sobre preguntas más complejas, mientras que un Router es más rígido al depender de reglas específicas. Además, se destaca que no requiere entrenamiento adicional, ya que el razonamiento pasa por el LLM. Por otro lado, el agente supervisor cuenta con la desventaja asociada al costo computacional, ya que usar un LLM como razonador es más costoso que utilizar una solución simplemente basada en reglas.

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [76]:
%pip install --upgrade --quiet gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.6 MB/s eta 0:00:00


Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [77]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = ... # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202",
    description="Hola! Soy un chatbot de aprendizaje de máquinas y estadística c:",
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://93ee8203a294149d4c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
